In [3]:
import urllib.request 
import re
from elasticsearch import Elasticsearch

In [104]:
import requests
# run bin/elasticsearch in elasticsearch directory in terminal before running this cell
res = requests.get('http://localhost:9200')

In [105]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [106]:
def test_ES(es):
    return es.ping()  # got True

In [107]:
if test_ES(es):
    print('ES instance working')
else:
    print('ES instance not working')

ES instance working


/home/mark/anaconda3/envs/nlp/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [108]:
def index_info(index_name):
    count, deleted, shards, =  es.cat.indices(index=index_name, h=['docs.count', 'docs.deleted', 'pri'])[:-1].split(' ')
    print(
      """
      #### INDEX INFO #####
      index_name = {}
      doc_count = {}
      shard_count = {}
      deleted_doc_count = {}
      """.format(index_name, count, shards, deleted)
  )

In [71]:
# read data from steam data directory 
games = pd.read_pickle('cleaned_steam_data')

In [86]:
#es.indices.delete(index='steam_test')

{'acknowledged': True}

In [87]:
request_body = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 1,
        
    },
    'mappings': {
          'properties': {
              'App_ID':{'type': 'long'},
              'title': {'type': 'text'},
              'abstract': {'type': 'text'},
              'body': {'type': 'text'},
              'developer': {'type': 'text'},
              'publisher': {'type': 'text'},
              'categories': {'type': 'text'},
              'genres': {'type': 'text'},
              'release_date': {'type': 'text'},
          }
    }
}

index_name = 'steam_test'
try:
    es.indices.get(index_name)
    print('index {} already exists'.format(index_name))
except:
    print('creating index {}'.format(index_name))
    es.indices.create(index_name, body=request_body)

creating index steam_test


In [92]:
for i, (title, App_ID, body, abstract, developer, publisher, categories, genres, release_date) in enumerate(games.values[:,1:]):
    doc_body = {
              'App_ID': App_ID,
              'title': title,
              'abstract': abstract,
              'body': body,
              'developer': developer,
              'publisher': publisher,
              'categories': categories,
              'genres': genres,
              'release_date': release_date
              }
    es.index(index_name, doc_body, id=i)

In [109]:
print('we have made and index called {} with {} documents'.format(index_name, es.cat.count(index=index_name,h=['count'])))
index_info(index_name)

we have made and index called steam_test with 56122
 documents

      #### INDEX INFO #####
      index_name = steam_test
      doc_count = 56122
      shard_count = 1
      deleted_doc_count = 0
      


In [111]:
es.get(index=index_name, id=50)

{'_index': 'steam_test',
 '_type': '_doc',
 '_id': '50',
 '_version': 1,
 '_seq_no': 50,
 '_primary_term': 1,
 'found': True,
 '_source': {'App_ID': 2290,
  'title': 'Final DOOM',
  'abstract': 'Two New, 32-Level DOOM II Episodes. Evilution: Far from earth, the UAC recommenced their experiments on on of the moons of Jupiter. A spaceship, mistaken for a supply vexxel on radar, hovered above the base. Hideous demons poured out, blanketing the base with death. All your comrades were quickly slaughtered or zombified.',
  'body': "Two New, 32-Level DOOM II Episodes.Evilution:Far from earth, the UAC recommenced their experiments on on of the moons of Jupiter. A spaceship, mistaken for a supply vexxel on radar, hovered above the base. Hideous demons poured out, blanketing the base with death. All your comrades were quickly slaughtered or zombified. This time, it's not about survival. It's about revenge.The Plutonia Experiment:Every effort has been made by the nation's top scientists to close 

In [95]:
query_body = {
    'query':{
        'term': {
            'body':  'ship'
        }
    }
}
print('### RESULTS ####')
explain=True
results = es.search(index=index_name, body=query_body, explain=explain)['hits']['hits']
for hit in results:
    print('title: {} - score: {}'.format(hit['_source']['title'], hit['_score']))
if explain:
    print('some info on results')
    print(hit['_explanation'])

### RESULTS ####
title: Poly Pirates - score: 6.499002
title: Bob Came in Pieces - score: 6.425233
title: Deep Space Battle Simulator - score: 6.351606
title: V ARRR - score: 6.2367477
title: 1912 Titanic Mystery - score: 6.2364287
title: SEX Cruise VR - score: 6.2364287
title: Rocket Squad - score: 6.191264
title: Captivus - score: 6.17612
title: Adrorium - score: 6.1709504
title: PXL-SHIP - score: 6.168356
some info on results
{'value': 6.168356, 'description': 'weight(body:ship in 41823) [PerFieldSimilarity], result of:', 'details': [{'value': 6.168356, 'description': 'score(freq=4.0), computed as boost * idf * tf from:', 'details': [{'value': 2.2, 'description': 'boost', 'details': []}, {'value': 3.2122538, 'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:', 'details': [{'value': 2257, 'description': 'n, number of documents containing term', 'details': []}, {'value': 56064, 'description': 'N, total number of documents with field', 'details': []}]}, {'value':